In [5]:
from arcgis.gis import GIS
import os
from multiprocessing import Process, Queue

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import sys
sys.path.append('../src')

import reach_tools

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
gis = GIS(username=os.getenv('ARCGIS_USERNAME'), password=os.getenv('ARCGIS_PASSWORD'))
gis

GIS @ https://knu2xs.maps.arcgis.com version:6.4

In [7]:
url_reach_line = os.getenv('URL_REACH_LINE')
url_reach_centroid = os.getenv('URL_REACH_CENTROID')
url_reach_points = os.getenv('URL_REACH_POINT')

yr_reach_line = reach_tools.ReachFeatureLayer(url_reach_line, gis)
lyr_reach_centroid = reach_tools.ReachFeatureLayer(url_reach_centroid, gis)
lyr_reach_points = reach_tools.ReachPointFeatureLayer(url_reach_points, gis)

In [11]:
def process_data(q):
    while True:
        reach_id = q.get()
        
        try:
            reach = reach_tools.Reach.get_from_aw(reach_id)
            reach.update_putin_takeout_and_trace()
            reach.publish(gis, lyr_reach_line, lyr_reach_centroid, lyr_reach_points)
            print('published', reach.reach_id, '-', reach.river_name, reach.reach_name)
        except:
            print('Could not publish', reach.reach_id, '-', reach.river_name, reach.reach_name)

In [ ]:
processed_df = lyr_reach_points.query(out_fields='reach_id', return_geometry=False).sdf
unprocessed_reach_id_lst = [val for val in range(1, 6001) if val not in processed_df.reach_id.astype('int64').values]

q = Queue()
processes = [Process(target=process_data, args=(q,)) for i in range(8)]

for p in processes:
    p.start()

for reach_id in unprocessed_reach_id_lst:
    q.put(reach_id)
    
for p in processes:
    p.join()

Parameters not valid for edit_features
Parameters not valid for edit_features
published 1967 - Mayo, N. Fork Va Route 629 to NC Route 1359
published 892 - Seavey Brook Long Lake to E. Machias R.
Parameters not valid for edit_features
published 1098 - Mayo, S. Fork VA Route 695 to Mayo River along NC Route 1359
Parameters not valid for edit_features
published 2028 - Adamson, W. Branch Route 100A to Putney
Parameters not valid for edit_features
published 1217 - Rio Chama 01\. Lobo Lodge to Chama River Park
Parameters not valid for edit_features
published 689 - Hoosic First Adams Dam to MA-5
published 2403 - Lost 2\. WV 55 Bridge 3.7 miles above Wardensville to WV 259 Bridge below
Wardensville
published 2405 - Meadow 1\. East Rainelle to Russellville
published 2404 - Manns Creek 2\. Confluence with Glade Creek to the Confluence with the New River
published 2406 - Meadow 2\. Nallen to Route 19 Bridge
published 2408 - Sleepy Creek, Meadow Branch Sleepy Creek Dam to bridge on CR9
published 2